In [36]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, recall_score

In [4]:
AAPL = pd.read_csv('../data/AAPL.csv')
noise_data = pd.read_csv('../data/noise_data.csv')

In [9]:
# Class == 1 : price goes up
# Class == 0 : price goes down
AAPL['Class'] = AAPL['Close'].shift(-1) < AAPL['Close']

In [88]:
data = pd.merge(AAPL[['Date','Class']], noise_data, on='Date', how='inner')
data

,Date,Class,sentiment_1,sentiment_2,sentiment_3,sentiment_4,sentiment_5,sentiment_6,sentiment_7,sentiment_8,sentiment_9,sentiment_10
0,2020-01-02,True,0.561438,0.475207,0.582919,0.456053,0.805911,0.407467,0.543495,0.404847,0.338051,0.594038
1,2020-01-03,False,0.495446,0.626114,0.659858,0.635979,0.636301,0.294901,0.654619,0.000000,0.000000,0.000000
2,2020-01-06,True,0.685909,0.531003,0.413658,0.599191,0.660934,0.525568,0.566146,0.592921,0.471915,0.567367
3,2020-01-07,False,0.612615,0.767221,0.780195,0.610177,0.638025,0.671396,0.453072,0.368407,0.551854,0.446610
4,2020-01-08,False,0.544420,0.342735,0.622565,0.696075,0.649117,0.357246,0.671396,0.568493,0.554695,0.623313
...,...,...,...,...,...,...,...,...,...,...,...,...
1022,2024-01-25,True,0.391377,0.464835,0.600638,0.393458,0.575343,0.611841,0.458810,0.551863,0.390478,0.447110
1023,2024-01-26,True,0.610214,0.308150,0.672705,0.597257,0.416289,0.752021,0.515445,0.483290,0.578394,0.614734
1024,2024-01-29,True,0.563793,0.528707,0.435842,0.563793,0.624860,0.664039,0.681199,0.501562,0.528563,0.641582
1025,2024-01-30,True,0.474414,0.668754,0.529539,0.714077,0.452377,0.405806,0.502249,0.511210,0.730571,0.352042


In [89]:
data['sentiment_sum'] = data['sentiment_1'] + data['sentiment_2'] + data['sentiment_3'] + data['sentiment_4'] + data['sentiment_5'] \
                        + data['sentiment_6'] + data['sentiment_7'] + data['sentiment_8'] + data['sentiment_9'] + data['sentiment_10']

In [90]:
data = data[['Date', 'sentiment_sum', 'Class']]
data['sentiment_sum'].plot(kind='hist', bins=10)

In [92]:
data['Date'] = pd.to_datetime(data['Date'])
data['Class'] = data['Class'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1027 entries, 0 to 1026
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           1027 non-null   datetime64[ns]
 1   sentiment_sum  1027 non-null   float64       
 2   Class          1027 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1)
memory usage: 28.1 KB


C:\Users\Xenrose\AppData\Local\Temp\ipykernel_28216\3715418551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = pd.to_datetime(data['Date'])
C:\Users\Xenrose\AppData\Local\Temp\ipykernel_28216\3715418551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Class'] = data['Class'].astype(int)


In [93]:
train = data[data['Date'] < '2023-08-01']
test = data[data['Date'] >= '2023-08-01']

X_train = train.drop(['Date', 'Class'], axis=1)
y_train = train['Class']

X_test = test.drop(['Date', 'Class'], axis=1)
y_test = test['Class']

In [94]:
data['Class'].value_counts()

0    538
1    489
Name: Class, dtype: int64

In [95]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
model.fit(X_train, y_train)

BernoulliNB()

In [96]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.51


In [97]:
recall_score(y_test, y_pred)

0.0

In [98]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.51      1.00      0.68        65
           1       0.00      0.00      0.00        62

    accuracy                           0.51       127
   macro avg       0.26      0.50      0.34       127
weighted avg       0.26      0.51      0.35       127



c:\Users\Xenrose\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Xenrose\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Xenrose\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

In [59]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [60]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [61]:
y_pred_lr = model.predict(X_test)
accuracy_score(y_test, y_pred_lr)

0.49606299212598426

In [101]:
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.50      0.91      0.65        65
           1       0.40      0.06      0.11        62

    accuracy                           0.50       127
   macro avg       0.45      0.49      0.38       127
weighted avg       0.45      0.50      0.39       127



In [103]:
y_pred_lr

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])